# Digital Signal Processing Parameter Development

This notebook is designed to determine the optimal parameters to use for the the experiment that ran 33 MeV deuterons on Ta in the vault on 25-27 April, 2017.  The experimental goals were:

1) Measure the input beam for the ETA. <br\>
2) Provide baseline response for each detector in the "native" environment. <br\>

This experiment ran 3 EJ-309 scintillator detectors located at 0, 45, and 90 degrees of the beam. 

This notebook describes all of the steps required to generate pile-up rejection parameters, integration windows, and PSD parameters.
 
The notebook is broken into the following sections: <br\>
1) Experiment Setup <br\>
2) Pile-up Rejection <br\>
3) Process Trees <br\>
4) PSD parameters <br\>
5) Convert to Scintillator Events <br\>
6) Make PHS Spectra <br\>
7) Perform Calibrations <br\>

At the start of each section, all of the modules necessary for that section are imported so that each can be run independently of the rest of the notebook (to the extent that the required inputs from previous sections exist). 

The notebook has dependencies on [PScripts](https://github.com/jamesbevins/PyScripts) and nsd-rootscripts.

## 1) Experimental Setup

### Initial Setup of DAQ

The DAQ was run in wave form mode. A separate pulse-amplitude-time data set was also taken, but it is not analyzed here.

### Detector Configuration

The detectors were set up as shown in the figure below (the figure is missing the detector at 45 and has an additional table):

<img src="../Pics/CleanBeamSetup.jpg">

The detectors were placed at the following coordinates.  These distances were measured to the face of the detector from the wall for horizontal detectors at 45 and 90, and from the center of the face of the vertical detector at 0 degrees.

EJ-309 (Det0): (81.0,0,0) cm <br\>
EJ-309 (Det45): (73.4,34.7,0) cm <br\>
EJ-309 (Det90): (52.4,91.4,0) cm <br\>

The connections were made as follows (no terminations were made on the output of the detectors):

Detector   | PMT Data Out | Junction Data | Junction HV | HV Ch # | DAQ Ch #  
---------- |:------------:|:-------------:|:-----------:|:-------:|:--------:
Det0       |A1            |30             |50 (HV1)     |0        |0
Det45      |A1            |31             |51 (HV2)     |1        |2
Det90      |A1            |32             |52 (HV3)     |2        |4

### Setting Configuration

#### Set Gain
To determine the neutron (proton) energy to MeVee conversion, lightTables.getLight(neutron_energy,particle) was used from the nsd-rootscripts repo.  For this calibration, the max neutron energy is ~38 MeV. The max Be neutron energy is ~ 11 MeV.  

With S = 1:  <br\>
lightTables.getLight(38,0) = 29.69 <br\>
lightTables.getLight(11,0) = 6.61  <br\>

For the 33 MeV on Ta, the ratio between the max E and the max AmBe neutron E is 4.5, which results in a FSR setting of 440 mV for neutrons, and the 4.4 MeV gamma line would be at 300 mV.  I am going to aim to put the edge of the gamma at 280 mV.  <br\>

**NOTE: Both the oscope and PSD must be scaled to ensure that the capture the full scale range (FSR) of the intended experiment.  There are two nobs to do this:**

1) Bias - The higher the bias the better the PSD [affects both] <br/>
2) Charge Sensitivity (on the channels tab) - Reduces the resolution, but allows for a wider FSR [only affects PSD]

#### Determine PSD Settings

After setting the gain the PSD gates and delays were set.  The best settings were determined to be:

Detector | DAQ Ch# | Bias | Charge Sens | Pre-trigger | Threshold | Record Length | Trigger Holdoff  
-------- |:-------:|:----:|:-----------:|:-----------:|:---------:|:-------------:|:----------------:
Det0     |0        |1235V |20           |40           |50         |496            |300
Det45    |2        |1250V |20           |40           |50         |496            |300
Det90    |4        |1250V |20           |40           |50         |496            |300

These are also saved as DppConfig_copy.txt in the 33MeVTa_25-27Apr17 folder on the DAQ and locally.


## 2) Pile-up Rejection

This code is best run interactively to assess the impact of the smoothing window and threshold on the pulse shape and detection of pile-up events.  

An example set of commands is provided for Channel 0 for the first 10000 events with a smoothing window of 9 and a threshold of 12.  It is recommended to run this set of commands for each channel of interest.

``` 
SCDigitalDaqPostProcessing ambe
ambe.loadFile("AmBe_002.0.root")
ambe.developPileUpRejectionParameters(0,10000,0,9,12)
```

After running the command, a graph will populate with a detected threshold event looking like this:

<img src="../Pics/Ch0_pileup.jpg">

The key here is to 1) ensure the baseline noise is smoothed without oversmoothing the peaks and 2) develop a threshold that captures real pulses without capturing random variation in the noise.  Yes, it is qualitative.  The final values used were:

Detector | DAQ Ch# | Smoothing | Threshold |
-------- |:-------:|:---------:|:---------:
Det0     |0        |7          |15         
Det45    |2        |7          |15         
Det90    |4        |7          |15         

## 3) Process Trees for Analysis 

The CFD Threshold was set to equal the software threshold @ 50 LSB.

The baseline samples were set at 16; ~32 available from pre-trigger.

The optimal integration window was chosen as 150, but multiple will be evaluated as part of the analysis process.

Each of the sub-files were then combined into a single file using "hadd" command:

```
hadd proc-ambe_002.root AmBe_002.0.root AmBe_002.1.root 
```

In [1]:
import sys
import os
from ROOT import gROOT

path = os.getcwd() + '/Calibration'
os.chdir(path)
print 'Currently working in: \n {}'.format(os.getcwd())

Currently working in: 
 /home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_25-27Apr17/WF/Calibration


In [2]:
gROOT.ProcessLine('SCDigitalDaqPostProcessing ambe')


47022921531432L

In [7]:
gROOT.ProcessLine('ambe.loadFile("AmBe_002.root")')
gROOT.ProcessLine('ambe.processTreesForTailToPeakAnalysis(“../Processed-Trees/proc-AmBe_002.root”,50)')

0L

This allows you to plot individual PSD plots by running commands such as:
    
``` 
DigPSDAnalysis psd
sd.loadFile("ambetree.root")
psd.getTailToPeakHist(0,16,28,16,200,-1)
TBrowser a
```

## 4) Determine Optimal PSD Parameters:

The optimal psd parameter for the tail to peak method are determined below.  Due to the interactive prompts, this section just describes the inputs used:

```
SCDigitalDaqPostProcessing test
test.loadFilepost.loadFile("CleanBeam_002.3.root")
test.calibrateDSP() 
```

The user prompts and respones: 

Are there any channels present that should be ignored while tuning?
(yes,no):
>yes

enter the channel number
>2

Is there another channel to ignore?
>yes

enter the channel number
>4

Is there another channel to ignore?
>yes

enter the channel number
>14

Is there another channel to ignore?
>no

What psd method would you like to use?
Tail to Peak: 1
Tail to Total: 2
90 10 Delta T: 3 
>1

Would you like to use a common start or one determined from a leading edge discriminator?
If you would like to use leading edge, enter -1*threshold in bits 
If you would like to use a common start please reference the plot and enter the sample number 
>-50

Enter Short Gate Min.(8)
>8

Enter Short Gate Max.(28)
>28

Enter Tail Min.(20)
>20

Enter Tail Max.(60)
>60

Enter tailOffsetMin.(6)
>6 

Enter tailOffsetMax.(24)
>24

Enter Integration Length(200)
>150

The results obtained were

The best figure of merit is 1.76518
The best parameters appear to be 
Peak Window = 12
Tail Window = 30
Tail Offset = 15

### Results
May permutations of threshold, integration widow, nd each window were tried.  In the end, there is a broad minimum that is best found with wide search parameters.  Although slightly better results could potentially be obtained by shifting the values by 1 or 2, this would have minimal impact on the final results.

Th summary of the results or each channel are:

#### Calibration - AmBe

Parameters      | Ch 0 | Ch 2 | Ch4 |
:--------------:|:----:|:----:|:----:
Threshold       |-50   |-50   |-50
Short Gate Min  |8     |10    |10         
Short Gate Max  |28    |15    |15         
Tail Min        |20    |24    |24  
Tail Max        |60    |36    |36
Tail Offset Min |6     |12    |12
Tail Offset Max |24    |18    |18
Integration Win |150   |150   |150
----------------|------|------|-----
FOM             |1.78  |1.75  |1.70
Peak Window     |12    |13    |14
Tail Window     |30    |33    |30
Tail Offset     |14    |14    |14

#### Beam

NOTE: Ch 2 and 4 did not return optimized results, presumably from a low number of neutron events due to being off-axis.  The Ch0 settings were copied, a reasonable approach given experience.  

Parameters      | Ch 0 | Ch 2 | Ch4 |
:--------------:|:----:|:----:|:----:
Threshold       |-50   |-50   |-50
Short Gate Min  |10    |8     |8         
Short Gate Max  |15    |28    |28         
Tail Min        |31    |20    |20  
Tail Max        |39    |60    |60
Tail Offset Min |10    |6     |6
Tail Offset Max |16    |24    |24
Integration Win |150   |150   |150
----------------|------|------|-----
FOM             |1.69  |      | 
Peak Window     |10    |10    |10
Tail Window     |31    |31    |31
Tail Offset     |13    |13    |13

#### ETA
Parameters      | Ch 0 | Ch 2 | Ch4 |
:--------------:|:----:|:----:|:----:
Threshold       |-50   |-50   |-50
Short Gate Min  |8     |10    |8         
Short Gate Max  |28    |15    |13         
Tail Min        |20    |24    |26  
Tail Max        |60    |36    |30
Tail Offset Min |6     |12    |12
Tail Offset Max |24    |18    |18
Integration Win |150   |150   |150
----------------|------|------|-----
FOM             |1.65  |1.67  |1.67
Peak Window     |10    |14    |11
Tail Window     |34    |30    |29
Tail Offset     |13    |12    |13


## 5)  Convert Waveforms to ScintillatorEvents

The following gROOT commands convert the Waveforms to Scintillator events according the the parameters determined above and set in 

```
calDSPFile.txt
```

The code below will loop over all of the calibration .roo files, convert to ScintillatorEvent .root trees, and output the code to add them to a combined tree.  The output code can be run from the command prompt in the directory with the .root files. 

In [1]:
import os
from ROOT import gROOT

runNum = ['002', '003']   #Run Numbers to include
fileNum = ['0', '1']

hadd= {}

os.chdir(os.getcwd()+'/Calibration/Raw')
print os.getcwd()

gROOT.ProcessLine('SCDigitalDaqPostProcessing post')

for filename in os.listdir(os.getcwd()):
    prefix = os.path.splitext(filename)[0]
    try:
        part = prefix.split('.')[1]
        run = prefix.split('.')[0].split('_')[1]
    except:
        part = -1
        run = -1
    
    if filename.endswith(".root") and run in runNum and part in fileNum:
        # Build dictionary off hadd commands
        name = prefix.split('.')[0]
        if name not in hadd:
            hadd[name] = 'hadd {}_SE.root {}_SE.root '.format(name, prefix)
        else:
            hadd[name] += '{}_SE.root '.format(prefix)
            
        # Convert to scintillator events
        print "Adding: ", filename
        gROOT.ProcessLine('post.loadFile("{}")'.format(filename))
        gROOT.ProcessLine('post.reduceToPATScintillatorEvent("{}_SE.root", "calibDSPFile.txt")'.format(prefix))

print "\n\nRun this code in the terminal from the following directory: {}\n".format(os.getcwd())
for k, v in hadd.iteritems():
    print v

/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_25-27Apr17/WF/Calibration/Raw
Adding:  Co60_003.1.root
Adding:  Cs137_002.0.root
Adding:  AmBe_002.1.root
Adding:  Co60_002.0.root
Adding:  Co60_003.0.root
Adding:  AmBe_002.0.root
Adding:  Cs137_002.1.root
Adding:  Co60_002.1.root
Adding:  AmBe_003.1.root
Adding:  Cs137_003.0.root
Adding:  AmBe_003.0.root


Run this code in the terminal from the following directory: /home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_25-27Apr17/WF/Calibration/Raw

hadd Cs137_003_SE.root Cs137_003.0_SE.root 
hadd Cs137_002_SE.root Cs137_002.0_SE.root Cs137_002.1_SE.root 
hadd AmBe_002_SE.root AmBe_002.1_SE.root AmBe_002.0_SE.root 
hadd AmBe_003_SE.root AmBe_003.1_SE.root AmBe_003.0_SE.root 
hadd Co60_003_SE.root Co60_003.1_SE.root Co60_003.0_SE.root 
hadd Co60_002_SE.root Co60_002.0_SE.root Co60_002.1_SE.root 


## 6)  Make Pulse Height Spectra

### AmBe Data (mixed neutron and gamma)

#### Gaussian fitted PSD cuts:

To perform the unfolding, it is necessary to toss out the gamma data from the target data.   A PSD cut based on a gaussian fitting of the neutron and gamma bands is used for the AmBe data

This cannot be run interactively since the software requires user inputs in the terminal and is a little finicky.  The process is described instead.  Each step lists the values used for each channel in the format (ch 0, ch1, ..., ch N)

First load up the file desired and specify the channel:

```
DDaqPostProcessing phs
phs.loadFile("AmBe_002_SE.root")

What is the tree name?
>channel_0_events

```

Next, create the necessary object and start the fit for the calibration set of interest:

```
phs.developFittedPSDCut(0)
```
This will open a PSD plot and ask for a linear PSD cut. Determine what linear cut works at high PH and enter it.  **(0.05, 0.052, ) used for this analysis**

It then asks for a PH above where mixing occurs.  Chose a PH channel where there is clearly no mixing.  **(8000, 10000) used for this analysis**

Next, enter a start bin # where there is PH data.  This is bin #, not channel #.  **(25, 25) used for this analysis**

Finally enter a stop bin number where the fitting will stop.  This is bin #, not channel #.  **(165, 205) used for this analysis**

This spawns a fitting window for the two distributions.  If the fit is good, hit update, close, and type a letter into the terminal to continue.  If the fit is bad, manually adjust the parameters until a decent fit is obtained and hit fit.  Then update, close, and type a letter in the terminal. 

The results should look something like this:

<img src="../Pics/FittedCuts_Ch2.png">

#### Linear low PH cuts:

The PHS will turn over due to the software threshold at some point.  This must be cut to avoid skewing the resulting unfolded neutron spectrum because the turnover is due to a software cutoff, not a lack of neutrons.  The following command will apply the Gaussian PSD cuts and ask for a linear PH cut **(1450, 1350 used for this analysis)**:

```
phs.makeDualPhSpectra("AmBe_002_ch0_PHS.root", true)
```

### Co60, Cs137 data

```
DDaqPostProcessing phs
phs.loadFile("Cs137_002_SE.root")
phs.getPhSpectrum()
TBrowser a
```

This will open a TBrowser window.  Right click on the PhSpectrum and select "save as"  'Cs137_002_ch0_PHS.root'

```
DDaqPostProcessing phs
phs.loadFile("Co60_002_SE.root")
phs.getPhSpectrum()
TBrowser a1
```

### Combine experimental data into a single file

**NOTE: Order matters!  This must be the same order as the data files above.** <br\>
**NOTE: These are run from the cmd line in the directory the files reside, not from within Root.**

```
rootcp AmBe_002_ch0_PHS.root:PHGamWithCut_0 Ch0_Calib.root:AmBe
rootcp Cs137_002_ch0_PHS.root:PhSpectrum_0 Ch0_Calib.root:Cs137
rootcp Co60_002_ch0_PHS.root:PhSpectrum_0 Ch0_Calib.root:Co60
```

### Combine the Calibration Simulation Files

**NOTE: Order matters!  This must be the same order as the data files above.** <br\>
**NOTE: These are run from the cmd line in the directory the files reside, not from within Root.**


```
rootcp AmBeDeposition.root:AmBeDeposition CalibSim.root:AmBeSim 
rootcp NaCsCoGammas.root:Cs137 CalibSim.root:Cs137Sim 
rootcp CoDeposition.root:CoDeposition CalibSim.root:Co60Sim 
```


### Rinse lather and repeat

For all channels....


## 7) Perform Calibrations

Simulation Based Calibrations

Required:  <br/>
1) Vector of pointers to histograms of data  <br/>
2) Vector of pointers to histograms of simulations of that data in the same order  <br/>

To run the calibrations, use the start16MeVTaCalibrations_detNum.cpp script located in the Calibration folder.  

**NOTE: The nsd-rootscripts repo contains an example that is similar. This script must be loaded first for both methods.**

This script contains user variables that must be modified.  For this mode, two methods are used and require modifications to the script:

1) startGuessSimCalib(): modify the file names <br/>
2) startSimCalibration(): modify the file names and each of the parameters  <br/>

It is recommended to use startGuessSimCalib() first; then use those parameters to use startSimCalibration with background subtraction.  However, if you have starting calibration and scale parameters, startSimCalibration() can be used without startGuessSimCalib(). 

### Detector 0 (ch0)

#### startGuessSimCalib

First load the script and run the method for detector 0:

In [1]:
import os
from ROOT import gROOT

path = '/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_25-27Apr17/WF/Calibration/ScintillatorEvents'
os.chdir(path)
print 'Currently working in: \n {}'.format(os.getcwd())

gROOT.ProcessLine('.L start33MeVTaCalibration_0.cpp')

Currently working in: 
 /home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_25-27Apr17/WF/Calibration/Raw


0L

Launch the gui:

In [2]:
gROOT.ProcessLine('obj0=startGuessSimCalib()')

108694512L

TPad::ResizePad:0: RuntimeWarning: Inf/NaN propagated to the pad. Check drawn objects.
TPad::ResizePad:0: RuntimeWarning: upperPad height changed from 0 to 10

TPad::ResizePad:0: RuntimeWarning: Inf/NaN propagated to the pad. Check drawn objects.
TPad::ResizePad:0: RuntimeWarning: lowerPad height changed from 0 to 10

TPad::ResizePad:0: RuntimeWarning: Inf/NaN propagated to the pad. Check drawn objects.
TPad::ResizePad:0: RuntimeWarning: upperPad height changed from 0 to 10

TPad::ResizePad:0: RuntimeWarning: Inf/NaN propagated to the pad. Check drawn objects.
TPad::ResizePad:0: RuntimeWarning: lowerPad height changed from 0 to 10

TPad::ResizePad:0: RuntimeWarning: Inf/NaN propagated to the pad. Check drawn objects.


Set the ranges and the good starting points using the GUI.

Then run the optimization routine in the cell below. 

Repeat the cell below until satified with the fit.

**NOTE: The plots should update.  From there, the ranges can be adjusted as necessary before running the optimization routine again. Ensure that no parameters are pegged to range limits.**

In [6]:
gROOT.ProcessLine('obj0->findGammaCalibrationConstants()')

159658096L

In [9]:
gROOT.ProcessLine('obj0->writeStateToDisk("CalibParams_0.txt")')

0L

#### startSimCalib

Perform a more thorough calibration using the parameters found from above (copy them from the calib_params file into the startSimCalib function).  A power law fit is used for the background subtraction.

In [1]:
import os
from ROOT import gROOT

path = '/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_25-27Apr17/WF/Calibration/ScintillatorEvents'
os.chdir(path)
print 'Currently working in: \n {}'.format(os.getcwd())

gROOT.ProcessLine('.L start33MeVTaCalibration_0.cpp')

Currently working in: 
 /home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_25-27Apr17/WF/Calibration/Raw


0L

In [2]:
gROOT.ProcessLine('sim0=startSimCalibration()')

119438176L

In [15]:
gROOT.ProcessLine('sim0->findGammaCalibrationConstants()')

705950624L

In [16]:
gROOT.ProcessLine('sim0->writeStateToDisk("CalibParams_0.txt")')

0L

In [17]:
gROOT.ProcessLine('sim0->getMinimizer()->Hesse()')

1L

The calibration is saved as **CalibParams_0.txt**.  The fit obtained was:

<img src="../Pics/33MeVTa_Det0FinalCalib.png">

##### The minimization results (Det0):

 ext. ||   Name    ||   type  ||     Value     ||  Error +/- 

   0   ||         a || limited ||   0.000250204069869 ||6.021956509881e-07 <br/>
   1   ||         b || limited ||    0.02117200623201 ||0.00156164423595 <br/>
   2   ||        Ec || limited ||     0.0922906401894 ||0.006951070557459 <br/>
   3   ||        E1 || limited ||     0.1619316922919 ||0.00251963153235 <br/>
   4   ||        E2 || limited ||    0.01000000314372* ||0.01556408847748 <br/>
   5   ||AmBeSim_scaler || limited ||      12515.79116676 ||115.5094235447 <br/>
   6   ||Cs137Sim_scaler || limited ||      183597.7173536 ||428.5828042477 <br/>
   7   ||Co60Sim_scaler || limited ||      66063.85137435 ||258.0181886081 <br/>

* Parameter is at Lower limit


covariance matrix status: 3 <br/>
covariance matrix:  <br/>
MnUserCovariance:  <br/>

  3.62642e-13 -9.22189e-10  3.82461e-09 -1.04584e-09 -7.01477e-16 -1.72787e-05   2.8022e-06  1.28513e-05 <br/>
 -9.22189e-10   2.4672e-06 -9.85076e-06  2.97065e-06  1.94802e-12    0.0436021  -0.00894099   -0.0304707 <br/>
  3.82461e-09 -9.85076e-06  4.86144e-05 -1.45475e-05 -2.86436e-12    -0.185787    0.0315942     0.137285 <br/>
 -1.04584e-09  2.97065e-06 -1.45475e-05  6.35064e-06 -7.56139e-12     0.050265   -0.0104243    -0.033823 <br/>
 -7.01477e-16  1.94802e-12 -2.86436e-12 -7.56139e-12    2.001e-10  3.11535e-08 -1.64882e-08  -2.0588e-08 <br/>
 -1.72787e-05    0.0436021    -0.185787     0.050265  3.11535e-08      13342.7      -129.03     -620.191 <br/>
   2.8022e-06  -0.00894099    0.0315942   -0.0104243 -1.64882e-08      -129.03       183685      65.0487 <br/>
  1.28513e-05   -0.0304707     0.137285    -0.033823  -2.0588e-08     -620.191      65.0487      66574.4 <br/>

MnUserCovariance Parameter correlations:  <br/>

            1    -0.974941     0.910888    -0.689154 -8.23476e-05    -0.248399    0.0108573    0.0827095 <br/>
    -0.974941            1    -0.899466     0.750483  8.76734e-05     0.240317   -0.0132815   -0.0751842 <br/>
     0.910888    -0.899466            1    -0.827936 -2.90417e-05    -0.230681    0.0105728    0.0763112 <br/>
    -0.689154     0.750483    -0.827936            1 -0.000212114     0.172678  -0.00965161   -0.0520175 <br/>
 -8.23476e-05  8.76734e-05 -2.90417e-05 -0.000212114            1  1.90661e-05 -2.71965e-06 -5.64077e-06 <br/>
    -0.248399     0.240317    -0.230681     0.172678  1.90661e-05            1  -0.00260635    -0.020809 <br/>
    0.0108573   -0.0132815    0.0105728  -0.00965161 -2.71965e-06  -0.00260635            1  0.000588231 <br/>
    0.0827095   -0.0751842    0.0763112   -0.0520175 -5.64077e-06    -0.020809  0.000588231            1 <br/>

global correlation coefficients :  <br/>
MnGlobalCorrelationCoeff:  <br/>

      0.98595 
     0.982836 
     0.956655 
     0.893037 
   0.00052918 
     0.248828 
    0.0169782 
    0.0864814 


In [18]:
# Plot Resolution function
gROOT.ProcessLine('TF1* resolution = sim0->getResolutionFunction()')
gROOT.ProcessLine('TCanvas c2')
gROOT.ProcessLine('resolution->Draw()')

0L

#### Resolution Function
<img src="../Pics/ResolutionFunct_Det0.png">

#### Resolution Function From Literature
<img src="../Pics/LitEJ309ResolutionFunct.png">

### Detector 45 (Ch2)
#### startSimCalib

Perform a more thorough calibration using the parameters found from above (copy them from the calib_params file into the startSimCalib function).  A power law fit is used for the background subtraction.

In [1]:
import os
from ROOT import gROOT

path = '/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_29-31Mar17/CalibData/'
os.chdir(path)
print 'Currently working in: \n {}'.format(os.getcwd())

gROOT.ProcessLine('.L start33MeVTaCalibration_45.cpp')

Currently working in: 
 /home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_29-31Mar17/CalibData


0L

In [2]:
gROOT.ProcessLine('sim0=startSimCalibration()')

101036864L

In [23]:
gROOT.ProcessLine('sim0->findGammaCalibrationConstants()')

107781200L

In [24]:
gROOT.ProcessLine('std::vector<double> params0 = sim0->findGammaCalibrationConstants()')
gROOT.ProcessLine('sim0->writeStateToDisk("CalibParams_2.txt")')

0L

In [25]:
gROOT.ProcessLine('sim0->getMinimizer()->Hesse()')

1L

The calibration is saved as **CalibParams_2.txt**.  The fit obtained was:

<img src="Pics/33MeVTa_Det45FinalCalib.png">

##### The minimization results (Det45):

 EIGENVALUES OF SECOND-DERIVATIVE MATRIX: <br\>
-2.5809e+02  4.3164e-01  7.2905e-01  8.0125e-01  9.1741e-01  9.3912e-01  9.4229e-01  9.9491e-01  9.9775e-01  1.0000e+00  1.0000e+00  1.0027e+00  1.0049e+00  1.0572e+00  1.2470e+00  1.6060e+00  2.6042e+02 <br\>
 FCN=2007.51 FROM HESSE     STATUS=NOT POSDEF    249 CALLS         250 TOTAL <br\>
                     EDM=9.25554e+06    STRATEGY= 1      ERR MATRIX NOT POS-DEF <br\>
  EXT PARAMETER                APPROXIMATE     INTERNAL      INTERNAL   <br\>
  NO.   NAME      VALUE            ERROR       STEP SIZE       VALUE    <br\>
   1  a            1.17689e-03   1.71843e-07   2.39372e-04   3.45520e-01 <br\>
   2  b            3.19963e-02   2.33971e-02   5.22846e-05  -2.29474e-04 <br\>
   3  Ec           1.22023e-01   7.49948e-06   1.08847e-02  -2.54454e-01 <br\>
   4  E1           9.75827e-02   1.00534e-05   1.65469e-02  -3.74239e-01 <br\>
   5  E2           1.52449e-01   7.06010e-05   3.28449e-03   1.08105e-01 <br\>
   
 EXTERNAL ERROR MATRIX.    NDIM=  25    NPAR= 17    ERR DEF=1 <br\>
 ELEMENTS ABOVE DIAGONAL ARE NOT PRINTED. <br\>
  2.953e-14 -5.382e-09 -4.518e-13 -1.009e-12 -1.194e-11 <br\>
 -5.382e-09  9.841e-04  8.261e-08  1.845e-07  2.184e-06  <br\>
 -4.518e-13  8.261e-08  5.624e-11  1.548e-11  1.833e-10 <br\>
 -1.009e-12  1.845e-07  1.548e-11  1.011e-10  4.092e-10 <br\>
 -1.194e-11  2.184e-06  1.833e-10  4.092e-10  4.984e-09  <br\>
ERR MATRIX NOT POS-DEF  <br\>

 PARAMETER  CORRELATION COEFFICIENTS   <br\>
       NO.  GLOBAL      1      2      3      4      5      6      7      8      9     10     11     12     13     14     15     16 <br\>
        1  0.99835   1.000 -0.998 -0.351 -0.584 -0.984 -0.498 -0.995 -0.524 -0.149 -0.305  0.000 -0.964  0.882 -0.002 -0.253 -0.092 <br\>
        2  0.99900  -0.998  1.000  0.351  0.585  0.986  0.499  0.996  0.525  0.149  0.306 -0.000  0.966 -0.883  0.002  0.254  0.092 <br\>
        3  0.35116  -0.351  0.351  1.000  0.205  0.346  0.175  0.350  0.184  0.052  0.107 -0.000  0.339 -0.310  0.001  0.089  0.032 <br\>
        4  0.58484  -0.584  0.585  0.205  1.000  0.577  0.292  0.583  0.307  0.087  0.179 -0.000  0.565 -0.517  0.001  0.148  0.054 <br\>
        5  0.98587  -0.984  0.986  0.346  0.577  1.000  0.492  0.982  0.517  0.147  0.301 -0.000  0.952 -0.871  0.002  0.250  0.091 <br\>
        6  0.49882  -0.498  0.499  0.175  0.292  0.492  1.000  0.497  0.262  0.074  0.152 -0.000  0.482 -0.441  0.001  0.127  0.046 <br\>
        7  0.99649  -0.995  0.996  0.350  0.583  0.982  0.497  1.000  0.523  0.149  0.305 -0.000  0.962 -0.880  0.002  0.253  0.092 <br\>
        8  0.52454  -0.524  0.525  0.184  0.307  0.517  0.262  0.523  1.000  0.078  0.160 -0.000  0.507 -0.463  0.001  0.133  0.048 <br\>
        9  0.14903  -0.149  0.149  0.052  0.087  0.147  0.074  0.149  0.078  1.000  0.045 -0.000  0.144 -0.132  0.000  0.038  0.014 <br\>
       10  0.30572  -0.305  0.306  0.107  0.179  0.301  0.152  0.305  0.160  0.045  1.000 -0.000  0.295 -0.270  0.000  0.078  0.028 <br\>
       11  0.00020   0.000 -0.000 -0.000 -0.000 -0.000 -0.000 -0.000 -0.000 -0.000 -0.000  1.000 -0.000  0.000 -0.000 -0.000 -0.000 <br\>
       12  0.96584  -0.964  0.966  0.339  0.565  0.952  0.482  0.962  0.507  0.144  0.295 -0.000  1.000 -0.853  0.002  0.245  0.089 <br\>
       13  0.88349   0.882 -0.883 -0.310 -0.517 -0.871 -0.441 -0.880 -0.463 -0.132 -0.270  0.000 -0.853  1.000 -0.001 -0.224 -0.082 <br\>
       14  0.00156  -0.002  0.002  0.001  0.001  0.002  0.001  0.002  0.001  0.000  0.000 -0.000  0.002 -0.001  1.000  0.000  0.000 <br\>
       15  0.25386  -0.253  0.254  0.089  0.148  0.250  0.127  0.253  0.133  0.038  0.078 -0.000  0.245 -0.224  0.000  1.000  0.023 <br\>
       16  0.09238  -0.092  0.092  0.032  0.054  0.091  0.046  0.092  0.048  0.014  0.028 -0.000  0.089 -0.082  0.000  0.023  1.000 <br\>
       17  0.01553  -0.016  0.016  0.005  0.009  0.015  0.008  0.015  0.008  0.002  0.005 -0.000  0.015 -0.014  0.000  0.004  0.001 <br\>

In [26]:
# Plot Resolution function
gROOT.ProcessLine('TF1* resolution = sim0->getResolutionFunction()')
gROOT.ProcessLine('TCanvas c2')
gROOT.ProcessLine('resolution->Draw()')

0L

#### Resolution Function
<img src="Pics/ResolutionFunct_Det45.png">

#### Resolution Function From Literature
<img src="Pics/LitEJ309ResolutionFunct.png">

### Detector 90 (Ch4)
#### startSimCalib

Perform a more thorough calibration using the parameters found from above (copy them from the calib_params file into the startSimCalib function).  A power law fit is used for the background subtraction.

In [7]:
import os
from ROOT import gROOT

path = '/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_29-31Mar17/CalibData/'
os.chdir(path)
print 'Currently working in: \n {}'.format(os.getcwd())

gROOT.ProcessLine('.L start33MeVTaCalibration_90.cpp')

Currently working in: 
 /home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/33MeVTa_29-31Mar17/CalibData


0L

In [8]:
gROOT.ProcessLine('sim0=startSimCalibration()')

120845008L

In [13]:
gROOT.ProcessLine('sim0->findGammaCalibrationConstants()')

123357472L

In [14]:
gROOT.ProcessLine('std::vector<double> params0 = sim0->findGammaCalibrationConstants()')
gROOT.ProcessLine('sim0->writeStateToDisk("CalibParams_4.txt")')

0L

In [15]:
gROOT.ProcessLine('sim0->getMinimizer()->Hesse()')

1L

The calibration is saved as **CalibParams_4.txt**.  The fit obtained was:

<img src="Pics/33MeVTa_Det90FinalCalib.png">

##### The minimization results (Det90):

 EIGENVALUES OF SECOND-DERIVATIVE MATRIX: <br\>
        -1.5225e-01  7.3387e-04  3.3268e-02  5.8975e-02  9.5601e-02  3.4520e-01  3.7284e-01  5.2547e-01  7.7016e-01  9.9462e-01  9.9542e-01  1.1758e+00  1.4002e+00  1.8223e+00  2.1730e+00  2.5915e+00  3.7971e+00 <br\>
 FCN=2418.41 FROM HESSE     STATUS=NOT POSDEF    247 CALLS         248 TOTAL <br\>
                     EDM=138.578    STRATEGY= 1      ERR MATRIX NOT POS-DEF <br\>
  EXT PARAMETER                APPROXIMATE     INTERNAL      INTERNAL  <br\> 
  NO.   NAME      VALUE            ERROR       STEP SIZE       VALUE   <br\> 
   1  a            1.04706e-03   2.24692e-07   2.19212e-04   8.07836e-02 <br\>
   2  b            3.33962e-02   1.33382e-04   4.22420e-03   8.73705e-02 <br\>
   3  Ec           1.17010e-01   1.25941e-03   1.26882e-02  -4.64887e-01 <br\>
   4  E1           1.20893e-01   2.21046e-03   7.14901e-03  -6.18203e-01 <br\>
   5  E2           1.38368e-01   1.08538e-03   1.37168e-02   9.34005e-01 <br\>
   
 EXTERNAL ERROR MATRIX.    NDIM=  25    NPAR= 17    ERR DEF=1
  5.049e-14 -1.854e-11  8.917e-11  3.237e-11  5.745e-12 <br\>
 -1.854e-11  1.779e-08 -3.362e-08 -6.442e-09  4.121e-08 <br\>
  8.917e-11 -3.362e-08  1.586e-06 -8.979e-07  8.388e-08 <br\>
  3.237e-11 -6.442e-09 -8.979e-07  4.887e-06 -1.594e-06 <br\>
  5.745e-12  4.121e-08  8.388e-08 -1.594e-06  1.178e-06 <br\> 

 PARAMETER  CORRELATION COEFFICIENTS   <br\>
       NO.  GLOBAL      1      2      3      4      5      6      7      8      9     10     11     12     13     14     15     16 <br\>
        1  0.78787   1.000 -0.618  0.315  0.065  0.024  0.167 -0.080  0.065  0.004  0.031  0.002  0.044 -0.066 -0.046 -0.013  0.054 <br\>
        2  0.79306  -0.618  1.000 -0.200 -0.022  0.285 -0.085  0.304  0.018 -0.001 -0.016 -0.001 -0.013  0.033  0.012 -0.002 -0.011 <br\>
        3  0.61076   0.315 -0.200  1.000 -0.322  0.061  0.215 -0.039  0.035 -0.149 -0.009  0.010 -0.046  0.019  0.046 -0.029  0.003 <br\>
        4  0.84430   0.065 -0.022 -0.322  1.000 -0.664  0.019  0.073  0.033 -0.010  0.001  0.001  0.146 -0.090 -0.152 -0.045 -0.022 <br\>
        5  0.84633   0.024  0.285  0.061 -0.664  1.000 -0.026  0.121  0.015  0.039  0.007 -0.002  0.129 -0.077 -0.138  0.017  0.025 <br\>
        6  0.65778   0.167 -0.085  0.215  0.019 -0.026  1.000 -0.002  0.018 -0.401 -0.000  0.024  0.009 -0.011 -0.010 -0.011  0.005 <br\>
        7  0.39669  -0.080  0.304 -0.039  0.073  0.121 -0.002  1.000  0.022 -0.003 -0.002  0.000  0.152 -0.078 -0.165 -0.009  0.001 <br\>
        8  0.67577   0.065  0.018  0.035  0.033  0.015  0.018  0.022  1.000 -0.005  0.002  0.000  0.013 -0.010 -0.014 -0.540  0.033 <br\>
        9  0.88075   0.004 -0.001 -0.149 -0.010  0.039 -0.401 -0.003 -0.005  1.000 -0.769 -0.028 -0.002  0.000  0.002  0.008  0.005 <br\>
       10  0.85332   0.031 -0.016 -0.009  0.001  0.007 -0.000 -0.002  0.002 -0.769  1.000 -0.016  0.001 -0.002 -0.001  0.001  0.002 <br\>
       11  0.07008   0.002 -0.001  0.010  0.001 -0.002  0.024  0.000  0.000 -0.028 -0.016  1.000  0.000 -0.000 -0.000 -0.001 -0.000 <br\>
       12  0.99666   0.044 -0.013 -0.046  0.146  0.129  0.009  0.152  0.013 -0.002  0.001  0.000  1.000 -0.577 -0.996 -0.017 -0.007 <br\>
       13  0.58256  -0.066  0.033  0.019 -0.090 -0.077 -0.011 -0.078 -0.010  0.000 -0.002 -0.000 -0.577  1.000  0.580  0.010  0.002 <br\>
       14  0.99676  -0.046  0.012  0.046 -0.152 -0.138 -0.010 -0.165 -0.014  0.002 -0.001 -0.000 -0.996  0.580  1.000  0.018  0.007 <br\>
       15  0.81604  -0.013 -0.002 -0.029 -0.045  0.017 -0.011 -0.009 -0.540  0.008  0.001 -0.001 -0.017  0.010  0.018  1.000  0.541 <br\>
       16  0.71760   0.054 -0.011  0.003 -0.022  0.025  0.005  0.001  0.033  0.005  0.002 -0.000 -0.007  0.002  0.007  0.541  1.000 <br\>
       17  0.70675  -0.002 -0.001  0.003  0.007 -0.005  0.001  0.000  0.107 -0.001 -0.000  0.000  0.002 -0.001 -0.003 -0.599 -0.626 <br\> 

In [16]:
# Plot Resolution function
gROOT.ProcessLine('TF1* resolution = sim0->getResolutionFunction()')
gROOT.ProcessLine('TCanvas c2')
gROOT.ProcessLine('resolution->Draw()')

0L

#### Resolution Function
<img src="Pics/ResolutionFunct_Det90.png">

#### Resolution Function From Literature
<img src="Pics/LitEJ309ResolutionFunct.png">

### To Dos:

 - Channel #2:
   - Cs to PHS
   - Co to PHS
   - Combine Exp Data Calibs
   - Calibration
   - Resolution Function
 - Channel #4:
   - AmBe to PHS
   - Cs to PHS
   - Co to PHS
   - Combine Exp Data Calibs
   - Calibration
   - Resolution Function